# 02 — Structs, Classes & Invariants 🏛️

## 1. The C View vs. The C++ View

In C, you view a struct as a **POD** (Plain Old Data). It is just a block of memory. Any code can modify any member at any time.

In C++, a class binds data and logic together to enforce **Invariants**.

### What is an Invariant?
An invariant is a rule about your data that must **always** be true. 
* Example: "A BankAccount balance cannot be negative."
* Example: "A Vector's size variable cannot be larger than its capacity."

If you allow public access to variables (`struct`), users can break invariants. If you use `class` and `private`, you can enforce them.


In [ ]:
#include <iostream>

// By default, 'class' members are private.
// By default, 'struct' members are public.
class BankAccount {
private:
    // Data is hidden. You cannot touch this directly from main().
    double balance;

public:
    // Constructor (Enforces initialization rules)
    BankAccount(double initial_deposit) {
        if (initial_deposit < 0) balance = 0;
        else balance = initial_deposit;
    }

    // Public Interface
    void deposit(double amount) {
        if (amount > 0) balance += amount;
    }

    double get_balance() const {
        return balance;
    }
};

{
    BankAccount account(100.0);
    // account.balance = -5000; // COMPILER ERROR! Private.

    account.deposit(50.0);
    std::cout << "Balance: " << account.get_balance() << std::endl;
}

## 2. Constructors vs `init()`

In C, you often write:

```c
struct System s;
system_init(&s);   // Don't forget this!
// ... use s ...
// If you forget init(), 's' contains garbage.
```

In C++, the **Constructor** guarantees initialization. You literally cannot create an uninitialized object of this class.

## 3. ⚠️ The `memset` Trap (CRITICAL)

As a C programmer, you love `memset(&obj, 0, sizeof(obj))` to zero out data.

**NEVER do this to a non-POD C++ object.**

Why?
1. **Internal Pointers:** `std::string` or `std::vector` manage pointers to heap memory. If you `memset` them to 0, you overwrite the address, causing memory leaks and crashes.
2. **V-Tables:** If a class has `virtual` functions, the object contains a hidden pointer (vptr) to a dispatch table. `memset` zeros this out, breaking polymorphism.

In [ ]:
#include <iostream>
#include <string>
#include <cstring> // for memset

struct C_Style_Struct {
    int id;
    float value;
};

struct Cpp_Class {
    std::string name;
    int id;
};

{
    // --- SAFE ---
    C_Style_Struct c_obj;
    std::memset(&c_obj, 0, sizeof(c_obj)); // Totally fine for PODs
    std::cout << "memset on POD: Safe." << std::endl;

    // --- DANGEROUS ---
    Cpp_Class cpp_obj;
    cpp_obj.name = "Hello";

    // UNCOMMENTING THE LINE BELOW WOULD CORRUPT THE STRING INTERNAL POINTERS
    // std::memset(&cpp_obj, 0, sizeof(cpp_obj));

    // In C++, we use Constructors or assignment to reset objects.
    cpp_obj = Cpp_Class{}; // Reassign a fresh empty object

    std::cout << "Reset C++ object safely." << std::endl;
}